<a href="https://colab.research.google.com/github/0xMalhotra/UAS-project/blob/main/UAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

# Load the input image
image = cv2.imread("input_image.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply adaptive thresholding to segment the image
thresh = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Iterate over each contour
# for contour in contours:
#     # Approximate the contour to a polygon
#     perimeter = cv2.arcLength(contour, True)
#     approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

#     # Classify the shape based on the number of vertices
#     if len(approx) == 3:
#         shape = "Triangle"
#     elif len(approx) == 4:
#         # Compute the bounding box to differentiate between squares and rectangles
#         (x, y, w, h) = cv2.boundingRect(approx)
#         aspect_ratio = w / float(h)
#         shape = "Square" if aspect_ratio >= 0.95 and aspect_ratio <= 1.05 else "Rectangle"
#     else:
#         shape = "Circle"

#     # Determine the location (land or ocean) based on the color of the region
#     mask = np.zeros(gray.shape, dtype="uint8")
#     cv2.drawContours(mask, [contour], -1, 255, -1)
#     mean_color = cv2.mean(image, mask=mask)[:3]

#     if mean_color[0] > 100:  # Blue channel corresponds to ocean
#         location = "Ocean"
#     else:
#         location = "Land"

#     # Draw the contour and label on the output image
#     cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
#     cv2.putText(
#         image,
#         f"{shape} ({location})",
#         (x, y - 10),
#         cv2.FONT_HERSHEY_SIMPLEX,
#         0.5,
#         (0, 255, 0),
#         2,
#     )

# Iterate over each contour
for contour in contours:
    # --- FIX STARTS HERE ---

    # First, get the bounding box for the current contour
    # This ensures x and y are defined for ALL shapes
    (x, y, w, h) = cv2.boundingRect(contour)

    # Now, approximate the contour to a polygon
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

    # --- FIX ENDS HERE ---

    # Classify the shape based on the number of vertices
    if len(approx) == 3:
        shape = "Triangle"
    elif len(approx) == 4:
        # We can now safely use w and h which were defined earlier
        aspect_ratio = w / float(h)
        shape = "Square" if aspect_ratio >= 0.95 and aspect_ratio <= 1.05 else "Rectangle"
    else:
        shape = "Circle"

    # Determine the location (land or ocean) based on the color of the region
    mask = np.zeros(gray.shape, dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_color = cv2.mean(image, mask=mask)[:3]

    if mean_color[0] > 100:  # Blue channel corresponds to ocean
        location = "Ocean"
    else:
        location = "Land"

    # Draw the contour and label on the output image
    cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
    cv2.putText(
        image,
        f"{shape} ({location})",
        (x, y - 10),  # This will now work for all shapes
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        2,
    )

# Save the output image
cv2.imwrite("output_image.png", image)

True

In [ ]:
import cv2
import numpy as np

def get_shape_templates():
    """
    Generates a dictionary of template contours for each shape.
    This creates a clean reference for our shape matching.
    """
    templates = {}

    # --- Star Template ---
    # A star is complex, so we define its points explicitly
    star_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 5], [61, 35], [95, 35], [68, 57], [79, 91], [50, 70], [21, 91], [32, 57], [5, 35], [39, 35]], np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.fillPoly(star_img, [pts], 255)
    contours, _ = cv2.findContours(star_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Child (Star)"] = contours[0]

    # --- Other Shape Templates ---
    # Circle
    circle_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.circle(circle_img, (50, 50), 45, 255, -1)
    contours, _ = cv2.findContours(circle_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Circle"] = contours[0]

    # Square
    square_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.rectangle(square_img, (10, 10), (90, 90), 255, -1)
    contours, _ = cv2.findContours(square_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Square"] = contours[0]

    # Rectangle
    rect_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.rectangle(rect_img, (10, 25), (90, 75), 255, -1)
    contours, _ = cv2.findContours(rect_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Rectangle"] = contours[0]

    # Triangle
    tri_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 10], [10, 90], [90, 90]], np.int32)
    cv2.fillPoly(tri_img, [pts], 255)
    contours, _ = cv2.findContours(tri_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Triangle"] = contours[0]

    return templates

# --- Main Script ---

# 1. Load Templates
shape_templates = get_shape_templates()

# 2. Load and Preprocess Input Image
try:
    image = cv2.imread("input_image.png")
    if image is None:
        raise FileNotFoundError("Error: Could not open or find the image.")
except Exception as e:
    print(e)
    exit()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 4)

kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)

# 3. Find Contours in the main image
contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
output_image = image.copy()

# 4. Loop, Match, and Classify
for contour in contours:
    if cv2.contourArea(contour) < 100:
        continue

    (x, y, w, h) = cv2.boundingRect(contour)

    # --- Shape Matching Logic ---
    min_score = float('inf')
    best_shape = "Unknown"

    for shape_name, template_contour in shape_templates.items():
        # Compare the contour from the image with each template
        match_score = cv2.matchShapes(contour, template_contour, cv2.CONTOURS_MATCH_I1, 0.0)

        if match_score < min_score:
            min_score = match_score
            best_shape = shape_name

    # --- Location and Drawing Logic ---
    mask = np.zeros(gray.shape, dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_color = cv2.mean(image, mask=mask)[:3]

    location = "Ocean" if mean_color[0] > 120 else "Land"

    cv2.drawContours(output_image, [contour], -1, (0, 255, 0), 2)
    cv2.putText(
        output_image,
        f"{best_shape} ({location})",
        (x, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        2
    )

cv2.imwrite("output_image_with_stars.png", output_image)
print("Processing complete. 'output_image_with_stars.png' has been saved.")

Processing complete. 'output_image_with_stars.png' has been saved.


In [ ]:
import cv2
import numpy as np
import math
import glob

# --- 1. CONFIGURATION AND MAPPINGS ---

# Define priority scores for casualties and emergencies
CASUALTY_SCORES = {"Child (Star)": 3, "Triangle": 2, "Square": 1, "Circle": 0} # Circle is pad
EMERGENCY_SCORES = {"red": 3, "yellow": 2, "green": 1} # Color-to-emergency mapping

# Define pad capacities and colors
PAD_CONFIG = {
    "pink": {"capacity": 3, "center": None, "assigned": [], "score_sum": 0},
    "blue": {"capacity": 4, "center": None, "assigned": [], "score_sum": 0},
    "grey": {"capacity": 2, "center": None, "assigned": [], "score_sum": 0}
}

# Define HSV color ranges for detection [Lower_Bound, Upper_Bound]
COLOR_RANGES = {
    # Pad Colors
    "blue": [np.array([90, 80, 50]), np.array([130, 255, 255])],
    "pink": [np.array([140, 80, 100]), np.array([170, 255, 255])],
    "grey": [np.array([0, 0, 50]), np.array([180, 30, 220])],
    # Casualty Colors
    "red": [np.array([0, 120, 70]), np.array([10, 255, 255])],
    "yellow": [np.array([25, 100, 100]), np.array([35, 255, 255])],
    "green": [np.array([40, 80, 50]), np.array([80, 255, 255])]
}

# --- 2. HELPER FUNCTIONS ---

def get_shape_templates():
    """Generates a dictionary of template contours for shape matching."""
    templates = {}
    # Star
    star_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 5], [61, 35], [95, 35], [68, 57], [79, 91], [50, 70], [21, 91], [32, 57], [5, 35], [39, 35]], np.int32)
    cv2.fillPoly(star_img, [pts.reshape((-1, 1, 2))], 255)
    contours, _ = cv2.findContours(star_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Child (Star)"] = contours[0]
    # Circle
    circle_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.circle(circle_img, (50, 50), 45, 255, -1)
    contours, _ = cv2.findContours(circle_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Circle"] = contours[0]
    # Square & Rectangle
    square_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.rectangle(square_img, (10, 10), (90, 90), 255, -1)
    contours, _ = cv2.findContours(square_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Square"] = contours[0]
    # Triangle
    tri_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 10], [10, 90], [90, 90]], np.int32)
    cv2.fillPoly(tri_img, [pts], 255)
    contours, _ = cv2.findContours(tri_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Triangle"] = contours[0]
    return templates

def get_centroid(contour):
    """Calculates the center (centroid) of a contour."""
    M = cv2.moments(contour)
    if M["m00"] == 0:
        return None
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    return (cx, cy)

def classify_shape(contour, templates):
    """Classifies a shape by matching it against templates."""
    min_score = float('inf')
    best_shape = "Unknown"
    for shape_name, template_contour in templates.items():
        score = cv2.matchShapes(contour, template_contour, cv2.CONTOURS_MATCH_I1, 0.0)
        if score < min_score:
            min_score = score
            best_shape = shape_name
    return best_shape

def euclidean_distance(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# --- 3. MAIN PROCESSING FUNCTION ---

def process_image(image_path, templates):
    """Main function to process a single image and return all required outputs."""
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    casualties = []
    pads = PAD_CONFIG.copy() # Reset pad config for each image

    # A. Detect all objects (casualties and pads)
    for color_name, (lower, upper) in COLOR_RANGES.items():
        mask = cv2.inRange(hsv_image, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            if cv2.contourArea(cnt) < 200: # Filter out noise
                continue

            centroid = get_centroid(cnt)
            if not centroid:
                continue

            shape = classify_shape(cnt, templates)

            if shape == "Circle" and color_name in pads: # It's a rescue pad
                pads[color_name]["center"] = centroid
            elif shape != "Circle" and color_name in EMERGENCY_SCORES: # It's a casualty
                casualty_score = CASUALTY_SCORES.get(shape, 0)
                emergency_score = EMERGENCY_SCORES.get(color_name, 0)
                casualties.append({
                    "id": len(casualties),
                    "shape": shape,
                    "color": color_name,
                    "center": centroid,
                    "casualty_score": casualty_score,
                    "emergency_score": emergency_score,
                    "priority_score": casualty_score * emergency_score
                })

    # B. Assignment Algorithm
    # Sort casualties by priority score (desc), then emergency score (desc)
    casualties.sort(key=lambda c: (c["priority_score"], c["emergency_score"]), reverse=True)

    for casualty in casualties:
        best_pad = None
        max_final_score = -1

        for pad_color, pad_info in pads.items():
            # Check if pad is detected and not full
            if pad_info["center"] and len(pad_info["assigned"]) < pad_info["capacity"]:
                dist = euclidean_distance(casualty["center"], pad_info["center"])
                if dist == 0: continue # Avoid division by zero

                # The custom final score for assignment decision
                final_score = (casualty["priority_score"]**2) / dist

                if final_score > max_final_score:
                    max_final_score = final_score
                    best_pad = pad_color

        if best_pad:
            pads[best_pad]["assigned"].append(casualty)
            pads[best_pad]["score_sum"] += casualty["priority_score"]

    # C. Generate Outputs
    # 1. Output Image (Land/Ocean Segmentation)
    land_ocean_mask = cv2.inRange(hsv_image, COLOR_RANGES["blue"][0], COLOR_RANGES["blue"][1])
    overlay = image.copy()
    overlay[land_ocean_mask != 0] = [255, 100, 100]  # Light blue for ocean
    overlay[land_ocean_mask == 0] = [100, 200, 100]  # Light green for land
    output_image = cv2.addWeighted(overlay, 0.5, image, 0.5, 0)

    # 2. Assigned Casualties Details
    assigned_details = {
        "blue": [[c["casualty_score"], c["emergency_score"]] for c in pads["blue"]["assigned"]],
        "pink": [[c["casualty_score"], c["emergency_score"]] for c in pads["pink"]["assigned"]],
        "grey": [[c["casualty_score"], c["emergency_score"]] for c in pads["grey"]["assigned"]]
    }

    # 3. Total Priority of Pads and Rescue Ratio
    pad_priorities = [pads["blue"]["score_sum"], pads["pink"]["score_sum"], pads["grey"]["score_sum"]]
    total_priority = sum(pad_priorities)
    rescue_ratio = total_priority / len(casualties) if casualties else 0

    return {
        "output_image": output_image,
        "assigned_details": assigned_details,
        "pad_priorities": pad_priorities,
        "rescue_ratio": rescue_ratio
    }

# --- 4. MAIN EXECUTION ---

if __name__ == "__main__":
    templates = get_shape_templates()
    image_files = glob.glob("input_image.png") # Assuming images are in a folder named 'input_images'

    results = []
    for image_file in image_files:
        result = process_image(image_file, templates)
        results.append({
            "image_name": image_file,
            "rescue_ratio": result["rescue_ratio"]
        })

        # Save output image
        output_filename = image_file.replace("input_images", "output_images")
        cv2.imwrite(output_filename, result["output_image"])

        # Print results for this image
        print(f"--- Results for {image_file} ---")
        print(f"1. Assigned Casualties: {result['assigned_details']}")
        print(f"2. Pad Priority Sums: {result['pad_priorities']}")
        print(f"3. Rescue Ratio (Pr): {result['rescue_ratio']:.2f}")
        print("-" * 30)

    # 4. Ranked Image List
    results.sort(key=lambda r: r["rescue_ratio"], reverse=True)
    ranked_images = [r["image_name"] for r in results]
    print("\n--- Final Image Ranking (by Rescue Ratio) ---")
    print(ranked_images)

--- Results for input_image.png ---
1. Assigned Casualties: {'blue': [[1, 1]], 'pink': [], 'grey': [[3, 1]]}
2. Pad Priority Sums: [1, 0, 3]
3. Rescue Ratio (Pr): 2.00
------------------------------

--- Final Image Ranking (by Rescue Ratio) ---
['input_image.png']


In [ ]:
import cv2
import numpy as np
import math
import glob
import os

# --- 1. CONFIGURATION AND MAPPINGS ---

# Define priority scores for casualties and emergencies
CASUALTY_SCORES = {"Child (Star)": 3, "Triangle": 2, "Square": 1, "Circle": 0}
EMERGENCY_SCORES = {"red": 3, "yellow": 2, "green": 1}

# Define pad capacities and colors
PAD_CONFIG = {
    "pink": {"capacity": 3, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (255, 0, 255)},
    "blue": {"capacity": 4, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (255, 0, 0)},
    "grey": {"capacity": 2, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (128, 128, 128)}
}

# Define HSV color ranges for detection [Lower_Bound, Upper_Bound]
COLOR_RANGES = {
    # Pad Colors
    "blue": [np.array([90, 80, 50]), np.array([130, 255, 255])],
    "pink": [np.array([140, 80, 100]), np.array([170, 255, 255])],
    "grey": [np.array([0, 0, 50]), np.array([180, 30, 220])],
    # Casualty Colors
    "red": [np.array([0, 120, 70]), np.array([10, 255, 255])],
    "yellow": [np.array([25, 100, 100]), np.array([35, 255, 255])],
    "green": [np.array([40, 80, 50]), np.array([80, 255, 255])]
}

# --- 2. HELPER FUNCTIONS ---

def get_shape_templates():
    # ... (This function remains the same as before)
    templates = {}
    star_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 5], [61, 35], [95, 35], [68, 57], [79, 91], [50, 70], [21, 91], [32, 57], [5, 35], [39, 35]], np.int32)
    cv2.fillPoly(star_img, [pts.reshape((-1, 1, 2))], 255)
    contours, _ = cv2.findContours(star_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Child (Star)"] = contours[0]
    circle_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.circle(circle_img, (50, 50), 45, 255, -1)
    contours, _ = cv2.findContours(circle_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Circle"] = contours[0]
    square_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.rectangle(square_img, (10, 10), (90, 90), 255, -1)
    contours, _ = cv2.findContours(square_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Square"] = contours[0]
    tri_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 10], [10, 90], [90, 90]], np.int32)
    cv2.fillPoly(tri_img, [pts], 255)
    contours, _ = cv2.findContours(tri_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Triangle"] = contours[0]
    return templates

def get_centroid(contour):
    # ... (This function remains the same)
    M = cv2.moments(contour)
    if M["m00"] == 0: return None
    return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

def classify_shape(contour, templates):
    # ... (This function remains the same)
    min_score, best_shape = float('inf'), "Unknown"
    for shape_name, template_contour in templates.items():
        score = cv2.matchShapes(contour, template_contour, cv2.CONTOURS_MATCH_I1, 0.0)
        if score < min_score:
            min_score, best_shape = score, shape_name
    return best_shape

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# --- 3. MAIN PROCESSING FUNCTION ---

def process_image(image_path, templates):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image at {image_path}. Skipping.")
        return None
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    casualties = []
    # Create a deep copy to avoid modifying the global config
    pads = {k: v.copy() for k, v in PAD_CONFIG.items()}

    # A. Detect all objects
    for color_name, (lower, upper) in COLOR_RANGES.items():
        mask = cv2.inRange(hsv_image, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv2.contourArea(cnt) < 150: continue
            centroid = get_centroid(cnt)
            if not centroid: continue
            shape = classify_shape(cnt, templates)
            if shape == "Circle" and color_name in pads:
                pads[color_name]["center"] = centroid
            elif shape != "Circle" and color_name in EMERGENCY_SCORES:
                cs, es = CASUALTY_SCORES.get(shape, 0), EMERGENCY_SCORES.get(color_name, 0)
                casualties.append({
                    "shape": shape, "color": color_name, "center": centroid,
                    "casualty_score": cs, "emergency_score": es, "priority_score": cs * es
                })

    # B. Assignment Algorithm & Drawing Logic
    casualties.sort(key=lambda c: (c["priority_score"], c["emergency_score"]), reverse=True)

    output_image = image.copy() # Use the original image as the base for drawing

    for casualty in casualties:
        best_pad, max_final_score = None, -1
        for pad_color, pad_info in pads.items():
            if pad_info.get("center") and len(pad_info.get("assigned", [])) < pad_info["capacity"]:
                dist = euclidean_distance(casualty["center"], pad_info["center"])
                if dist < 1: continue
                final_score = (casualty["priority_score"]**2) / dist
                if final_score > max_final_score:
                    max_final_score, best_pad = final_score, pad_color

        if best_pad:
            pads[best_pad]["assigned"].append(casualty)
            pads[best_pad]["score_sum"] = pads[best_pad].get("score_sum", 0) + casualty["priority_score"]

            # --- VISUALIZATION DRAWING ---
            pad_center = pads[best_pad]["center"]
            pad_draw_color = pads[best_pad]["color_bgr"]
            # Draw line from casualty to pad
            cv2.line(output_image, casualty["center"], pad_center, pad_draw_color, 2)
            # Draw a circle on the casualty for emphasis
            cv2.circle(output_image, casualty["center"], 15, pad_draw_color, 2)

    # C. Generate Outputs
    assigned_details = {
        "blue": [[c["casualty_score"], c["emergency_score"]] for c in pads["blue"].get("assigned", [])],
        "pink": [[c["casualty_score"], c["emergency_score"]] for c in pads["pink"].get("assigned", [])],
        "grey": [[c["casualty_score"], c["emergency_score"]] for c in pads["grey"].get("assigned", [])]
    }
    pad_priorities = [pads["blue"].get("score_sum", 0), pads["pink"].get("score_sum", 0), pads["grey"].get("score_sum", 0)]
    total_priority = sum(pad_priorities)
    rescue_ratio = total_priority / len(casualties) if casualties else 0

    return {
        "output_image": output_image,
        "assigned_details": assigned_details,
        "pad_priorities": pad_priorities,
        "rescue_ratio": rescue_ratio
    }

# --- 4. MAIN EXECUTION ---

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("input_images", exist_ok=True)
    os.makedirs("output_images", exist_ok=True)

    templates = get_shape_templates()
    # IMPORTANT: Place your 10 input images in a folder named 'input_images'
    image_files = glob.glob("input_images/*.png") + glob.glob("input_images/*.jpg")

    if not image_files:
        print("ERROR: No images found in the 'input_images' folder. Please add your images and try again.")

    all_results = []
    for image_file in image_files:
        result = process_image(image_file, templates)
        if result:
            all_results.append({"image_name": os.path.basename(image_file), "rescue_ratio": result["rescue_ratio"]})

            # Save the visual output image
            output_filename = os.path.join("output_images", f"output_{os.path.basename(image_file)}")
            cv2.imwrite(output_filename, result["output_image"])

            print(f"--- Results for {os.path.basename(image_file)} ---")
            print(f"1. Output image saved to: {output_filename}")
            print(f"2. Assigned Casualties (by pad): {result['assigned_details']}")
            print(f"3. Pad Priority Sums [Blue, Pink, Grey]: {result['pad_priorities']}")
            print(f"4. Rescue Ratio (Pr): {result['rescue_ratio']:.2f}\n")

    # Final ranked list of images
    all_results.sort(key=lambda r: r["rescue_ratio"], reverse=True)
    ranked_images = [r["image_name"] for r in all_results]
    print("--- FINAL RANKING ---")
    print(f"4. Images ranked by Rescue Ratio (descending): {ranked_images}")

--- Results for input_image.png ---
1. Output image saved to: output_images/output_input_image.png
2. Assigned Casualties (by pad): {'blue': [[3, 1], [3, 1], [2, 1], [2, 1]], 'pink': [], 'grey': []}
3. Pad Priority Sums [Blue, Pink, Grey]: [10, 0, 0]
4. Rescue Ratio (Pr): 2.00

--- FINAL RANKING ---
4. Images ranked by Rescue Ratio (descending): ['input_image.png']
